In [1]:
import re
import os
import pandas as pd
import numpy as np
import traceback2
from datetime import datetime
import pymysql

where_map = {}
# pairs regexp: "\[([a-zA-z0-9$\-+τ\.%]*)\]-", "-\[([a-zA-z0-9$\-+τ]*)\]"
ignorable_files = ["yield_data_2021_06_07.csv","Alternate.csv","vfat_tools_data_2021_06_07.csv",'.ipynb_checkpoints', "Yield Farm Manual 6:7 New .xlsx", "crash_parser.ipynb", "yieldFarm_6-7.csv"]

In [2]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()

In [3]:
def get_raydium_data(date):
    sql = f'select * from raydium_farm_pool_data where timestamp_dt = FROM_UNIXTIME({date});'
    data = pd.read_sql_query(sql, __connect__())
    data['apr'] = data['pool_rate']
    data['network'] = 'SOL'
    data['type_of_agriculture'] = data['farm_payment'].apply(lambda x : 'FARM' if x is not None else 'POOL')
    return data[['pair1', 'pair2', 'apr', 'where', 'network', 'type_of_agriculture']]

def get_liquidityfolio_pool_data(date):
    sql = f'select * from liquidityfolio_pool where timestamp_dt = FROM_UNIXTIME({date}) and `where` != "balancer";'
    data = pd.read_sql_query(sql, __connect__())
    data['apr'] = data['pool_rate'] * 100 * 12
    data['type_of_agriculture'] = data['farm_payment'].apply(lambda x : 'FARM' if x is not None else 'POOL')
    data['network'] = 'ETH'
    return data[['pair1', 'pair2', 'apr', 'where', 'network', 'type_of_agriculture']]

In [4]:
date = datetime(2021, 6, 7)
dt = date.timestamp()

In [5]:
df = get_raydium_data(dt)
df = df.append(get_liquidityfolio_pool_data(dt))
df[df['where'] == 'balancer']

,pair1,pair2,apr,where,network,type_of_agriculture


In [6]:
sm = {
    "WBTC" : "BTC",
    "WETH" : "ETH",
    "WBNB" : "BNB",
    "WFTM" : "FTM",
    "WAVAX" : "AVAX",
}


In [7]:
def pair1(line):
#     print(line)
    try:
        is_3_pairs = line.split("]-[")
        if len(is_3_pairs) > 2:
            return None
        s_index = line.index("[") + 1
        s = line[s_index: line.index("]", s_index)]
        if all(ord(c) < 128 for c in s):
            return s
        else:
            print(line)
            return None
    except:
        return ''

def pair2(line):
    print(line)
    s_index = line.index("-[") + 2
    s = line[s_index: line.index("]", s_index)]
#     print(s)
    if all(ord(c) < 128 for c in s):
        return s
    else:
        print(line)
        return ''

In [8]:
def apr(line):
    apr = "Year [0-9\,\.]+\%"
    x = re.findall(apr, line)
    return x[0].replace("Year ", "").replace("%", "").strip() if len(x) else 0

In [9]:
def tvl(line):
    tvl = "TVL: \$[0-9\.\,]+"
    x = re.findall(tvl, line)
    return x[0].replace("TVL: $", "").replace("%", "").replace(",","").strip()

In [10]:
def where(line):
    if 'INFO' in line:
        return line.replace("INFO", "").replace('\n', '').replace(":", "").replace("//", "://").strip()
    return None

In [11]:
def get_single_pair(line):
    exp = "\d* *.*Price:"
    x = re.findall(exp, line)
    if not len(x):
#         print(">>>>>>>>>>>",line)
        return
    pair = x[0].replace(" Price:", "")
    return pair[pair.rindex(" "):].strip() if " " in pair else pair

def get_single_tvl(line):
    exp = "\([0-9\,\.$]+\)"
    x = re.findall(exp, line)
    return x[0].replace("(", "").replace(")", "").replace("$", "").replace(",","").strip()


In [12]:
def read_blocks_from_files(file):
    f = open(file)
    lines = f.readlines()
    blocks = []
    block = []
    for line in lines:
        line = line.strip()
        if line == "" or len(re.findall("INFO[ ]+: ", line)):
            blocks.append(block)
            if len(line):
                blocks.append([line])
            block = []
        else:
            block.append(line)
    blocks.append(block)
    return blocks

def get_pair_info_from_block(block, wh, network):
    tvl_index = 0
    if network == 'BSC':
        tvl_index = 3
    if len(block) == 1:
        if 'INFO' in block[0]:
            return [where(block[0])]
        else:
            return [wh]
    if len(block) and pair1(block[0]):
        apr_index = -1
        for index in range(1, len(block)):
            if 'APR: ' in block[index]:
                apr_index = index
                break
        return [pair1(block[0]), pair2(block[0]), get_single_tvl(block[tvl_index]) if tvl_index > 0 else tvl(block[tvl_index]), apr(block[apr_index]) if apr_index > 1 else 0, where_map[wh] if wh in where_map else wh, network]
    elif len(block) > 2 and get_single_pair(block[0]):
        apr_index = -1
        for index in range(1, len(block)):
            if 'APR: ' in block[index]:
                apr_index = index
                break
        return [get_single_pair(block[0]), "", get_single_tvl(block[1]), apr(block[apr_index]) if apr_index > 1 else 0, where_map[wh] if wh in where_map else wh, network]
        
    return None


def get_pairs(blocks, network):
    where = ""
    pairs = []
    for block in blocks:
        try:
            pair = get_pair_info_from_block(block, where, network)
            print(pair)
            if len(where) == 0 and pair and len(pair) > 2:
                print(block, network)
            if pair:
                if len(pair) == 1 and pair[0] and len(pair[0]):
                    where = pair[0]
                else:
                    pairs.append(pair)
        except Exception:
            print(block[0])
    return pairs
        

In [13]:
data_dir = './'
def conver_files_to_one_df(data_dir):
    all_pairs = []
    files = os.listdir(data_dir)
    for file_name in files:
        if file_name in ignorable_files:
            continue
        network = file_name[0:file_name.index(" ")]
        if 'Alternate' == network:
            network = 'ETH'
#         if 'BSC' != network:
#             continue
        print(file_name)
        blocks = read_blocks_from_files(f"{data_dir}{file_name}")
        pairs = get_pairs(blocks, network)
        all_pairs.extend(pairs)
    return all_pairs

In [14]:
pairs = conver_files_to_one_df(data_dir)

FANTOM 6:7 Yields.crash
None
['https://popsicle.finance']
['https://popsicle.finance']
None
0 - [WFTM]-[ICE] SLP [+] [-] [<=>] Price: $1.91 TVL: $3,425,040.24
['WFTM', 'ICE', '3425040.24', '219.04', 'https://popsicle.finance', 'FANTOM']
1 - [WFTM]-[ICE] SPOOKY LP [+] [-] [<=>] Price: $1.79 TVL: $3,032,060.91
['WFTM', 'ICE', '3032060.91', '239.93', 'https://popsicle.finance', 'FANTOM']
['https://popsicle.finance']
['ICE', '', '155258.78', '0.00', 'https://popsicle.finance', 'FANTOM']
1 - [WFTM]-[ICE] SLP [+] [-] [<=>] Price: $1.91 TVL: $3,425,040.24
['WFTM', 'ICE', '3425040.24', '0.00', 'https://popsicle.finance', 'FANTOM']
['fUSDT+DAI+USDC', '', '27704.13', '0.00', 'https://popsicle.finance', 'FANTOM']
['https://popsicle.finance']
None
None
['https://spookyswap.finance']
['https://spookyswap.finance']
0 - [WFTM]-[BOO] SPOOKY LP [+] [-] [<=>] Price: $2.70 TVL: $10,163,940.14
['WFTM', 'BOO', '10163940.14', '200.84', 'https://spookyswap.finance', 'FANTOM']
1 - [fUSDT]-[WFTM] SPOOKY LP [+]

['WBNB', 'BUSD', '59067.16', '90.94', 'https://pool.mochiswap.io', 'BSC']
8 - [ADA]-[WBNB] Uni LP [+] [-] [<=>] Price: $51.94 TVL: $47,781.99
['ADA', 'WBNB', '47731.55', '112.54', 'https://pool.mochiswap.io', 'BSC']
9 - [DOT]-[WBNB] Uni LP [+] [-] [<=>] Price: $195.61 TVL: $72,071.81
['DOT', 'WBNB', '70728.48', '75.95', 'https://pool.mochiswap.io', 'BSC']
10 - [ETH]-[WBNB] Uni LP [+] [-] [<=>] Price: $2,087.60 TVL: $44,082.64
['ETH', 'WBNB', '44044.50', '121.96', 'https://pool.mochiswap.io', 'BSC']
12 - [BTCB]-[WBNB] Uni LP [+] [-] [<=>] Price: $7,544.68 TVL: $44,151.04
['BTCB', 'WBNB', '44103.63', '121.79', 'https://pool.mochiswap.io', 'BSC']
13 - [DOGE]-[WBNB] Uni LP [+] [-] [<=>] Price: $2,491,169.35 TVL: $37,006.72
['DOGE', 'WBNB', '36967.99', '124.55', 'https://pool.mochiswap.io', 'BSC']
14 - [MOCHI]-[APESOX] Uni LP [+] [-] [<=>] Price: $9.80 TVL: $58,815.51
['MOCHI', 'APESOX', '58227.26', '168.69', 'https://pool.mochiswap.io', 'BSC']
15 - [APESOX]-[WBNB] Uni LP [+] [-] [<=>] Pric

31 - [ETH]-[WBNB] Cake LP [+] [-] [<=>] Price: $2,122.47 TVL: $186,831,441.08
['ETH', 'WBNB', '5529.20', '175.41', 'https://vanillacake.farm', 'BSC']
['https://vanillacake.farm']
None
None
['https://garudaswap.finance']
['https://garudaswap.finance']
0 - [WBNB]-[BUSD] Cake LP [+] [-] [<=>] Price: $43.06 TVL: $333,984,492.66
['WBNB', 'BUSD', '130755.54', '75.37', 'https://garudaswap.finance', 'BSC']
1 - [BTCB]-[WBNB] Cake LP [+] [-] [<=>] Price: $7,676.73 TVL: $130,962,405.53
['BTCB', 'WBNB', '164367.10', '47.97', 'https://garudaswap.finance', 'BSC']
2 - [ETH]-[WBNB] Cake LP [+] [-] [<=>] Price: $2,122.47 TVL: $186,831,450.52
['ETH', 'WBNB', '867750.29', '6.81', 'https://garudaswap.finance', 'BSC']
3 - [DOT]-[WBNB] Cake LP [+] [-] [<=>] Price: $197.07 TVL: $34,450,414.28
['DOT', 'WBNB', '126322.42', '31.21', 'https://garudaswap.finance', 'BSC']
4 - [USDT]-[WBNB] Cake LP [+] [-] [<=>] Price: $43.38 TVL: $135,533,037.33
['USDT', 'WBNB', '37960.14', '103.85', 'https://garudaswap.finance', 

['https://app.elk.finance']
None
None
['https://polycat.finance']
['https://polycat.finance']
['UNI-V2', '', '0.00', 0, 'https://polycat.finance', 'MATIC']
['FISH', '', '0.00', 0, 'https://polycat.finance', 'MATIC']
['WMATIC', '', '18029913.09', 0, 'https://polycat.finance', 'MATIC']
['WBTC', '', '14780281.05', 0, 'https://polycat.finance', 'MATIC']
['WETH', '', '24005005.04', 0, 'https://polycat.finance', 'MATIC']
['QUICK', '', '5409097.74', 0, 'https://polycat.finance', 'MATIC']
['USDT', '', '10978846.68', 0, 'https://polycat.finance', 'MATIC']
['USDC', '', '11834352.82', 0, 'https://polycat.finance', 'MATIC']
['DAI', '', '11120861.87', 0, 'https://polycat.finance', 'MATIC']
['Krill', '', '68411.83', 0, 'https://polycat.finance', 'MATIC']
['LINK', '', '6317914.02', 0, 'https://polycat.finance', 'MATIC']
['PolyDoge', '', '9140822.11', 0, 'https://polycat.finance', 'MATIC']
['SUSHI', '', '8303977.76', 0, 'https://polycat.finance', 'MATIC']
['SLP', '', '0.00', 0, 'https://polycat.financ

In [15]:
pair_df = pd.DataFrame(pairs, columns=['pair1', 'pair2', 'tvl', 'apr', 'where', 'network'])
pair_df['apy'] = np.nan

In [16]:
pairs

[['WFTM', 'ICE', '3425040.24', '219.04', 'https://popsicle.finance', 'FANTOM'],
 ['WFTM', 'ICE', '3032060.91', '239.93', 'https://popsicle.finance', 'FANTOM'],
 ['ICE', '', '155258.78', '0.00', 'https://popsicle.finance', 'FANTOM'],
 ['WFTM', 'ICE', '3425040.24', '0.00', 'https://popsicle.finance', 'FANTOM'],
 ['fUSDT+DAI+USDC',
  '',
  '27704.13',
  '0.00',
  'https://popsicle.finance',
  'FANTOM'],
 ['WFTM',
  'BOO',
  '10163940.14',
  '200.84',
  'https://spookyswap.finance',
  'FANTOM'],
 ['fUSDT',
  'WFTM',
  '3371622.58',
  '95.02',
  'https://spookyswap.finance',
  'FANTOM'],
 ['USDC',
  'WFTM',
  '4037212.83',
  '79.57',
  'https://spookyswap.finance',
  'FANTOM'],
 ['WFTM',
  'DAI',
  '3240635.33',
  '99.27',
  'https://spookyswap.finance',
  'FANTOM'],
 ['WFTM',
  'BTC',
  '3221214.10',
  '79.44',
  'https://spookyswap.finance',
  'FANTOM'],
 ['WFTM',
  'ETH',
  '4336543.18',
  '61.47',
  'https://spookyswap.finance',
  'FANTOM'],
 ['WFTM',
  'LINK',
  '1850222.03',
  '69.01'

In [17]:
pair_df

,pair1,pair2,tvl,apr,where,network,apy
0,WFTM,ICE,3425040.24,219.04,https://popsicle.finance,FANTOM,NaN
1,WFTM,ICE,3032060.91,239.93,https://popsicle.finance,FANTOM,NaN
2,ICE,,155258.78,0.00,https://popsicle.finance,FANTOM,NaN
3,WFTM,ICE,3425040.24,0.00,https://popsicle.finance,FANTOM,NaN
4,fUSDT+DAI+USDC,,27704.13,0.00,https://popsicle.finance,FANTOM,NaN
...,...,...,...,...,...,...,...
2563,FUSE,USDC,78652.58,279.02,https://fuse.io,ETH,NaN
2564,FUSE,WETH,1025341.98,129.89,https://fuse.io,ETH,NaN
2565,WILD,WETH,895056.11,1700.00,https://wild.credit,ETH,NaN
2566,SPELL,WETH,12514487.48,195.06,https://abracadabra.money,ETH,NaN


In [18]:
pair_df[pair_df['network'] == 'Alternate']['network'] = 'ETH'

In [19]:
sheet_pairs_df = pd.read_csv('yieldFarm_6-7.csv')
eth_sheet_pairs_df = pd.read_csv('Alternate.csv')
# eth_sheet_pairs_df.columns=['pair1', 'pair2', 'tvl', 'where', 'apr']
eth_sheet_pairs_df['apy'] = np.nan
eth_sheet_pairs_df['network'] = 'ETH'
sheet_pairs_df = sheet_pairs_df.append(eth_sheet_pairs_df)
sheet_pairs_df['pair2'] = sheet_pairs_df['pair2'].fillna('')

In [20]:
sheet_pairs_df

,pair1,pair2,apr,apy,tvl,where,network
0,MEATBALL,BUSD,"12,573.73",NaN,515,https://swedefi.com/,BSC
1,MEATBALL,BNB,"6,924.99",NaN,818,https://swedefi.com/,BSC
2,BNB,BUSD,338.72,NaN,"2,389",https://swedefi.com/,BSC
3,CAKE,BNB,214.74,NaN,"3,768",https://swedefi.com/,BSC
4,USDT,BUSD,"1,116.39",NaN,290,https://swedefi.com/,BSC
...,...,...,...,...,...,...,...
190,kmSNX,USDC,NaN,NaN,0.0,SUSHI,ETH
191,kmRUNE,sUSD,NaN,NaN,0.0,SUSHI,ETH
192,kmREN,USDC,NaN,NaN,0.0,SUSHI,ETH
193,kmAAVE,DAI,NaN,NaN,0.0,SUSHI,ETH


In [21]:
def update_apr(apr):
    apr = f"{apr}".replace("%", "")
    if len(apr) == 0 or "nan" == apr:
        return np.nan
    else:
        return apr

In [22]:
pair_df = pair_df.append(sheet_pairs_df).reset_index(drop=True)
pair_df['type_of_agriculture'] = 'FARM'
pair_df = pair_df.append(df).reset_index(drop=True)





pair_df['apr'] = pair_df['apr'].apply(lambda x: update_apr(x))
pair_df['pair1'] = pair_df['pair1'].apply(lambda x: sm[x.strip()] if x.strip() in sm else x.strip())
pair_df['pair2'] = pair_df['pair2'].apply(lambda x: sm[x.strip()] if x.strip() in sm else x.strip())

In [23]:
pair_df['pair2'].unique()

array(['ICE', '', 'BOO', 'FTM', 'DAI', 'BTC', 'ETH', 'LINK', 'AAVE',
       'SNX', 'SUSHI', 'BAND', 'CREAM', 'COVER', 'BADGER', 'BNB', 'WOOFY',
       'ANY', 'BIFI', 'SPIRIT', 'YFI', 'FRAX', 'BITB', 'CzTears', 'SUPRA',
       'MM', 'BUSD', 'GTON', 'SON', 'BORG', 'AURORA', 'BORGDAI', 'FUSD',
       'OPE', 'UBI', 'ORI', 'ESWAP', 'FROYO', 'GHOST', 'LQDR', 'WAKA',
       'ATRI', 'ELK', 'DSTEIN', 'BRUSH', 'TOMB', 'TSHARE', 'AVAX', 'USDT',
       'UNI', 'PNG', 'BAMBOO-V2', 'SNOB', 'PEFI', 'GDL', 'MKC', 'NEKO',
       'XAVA', 'CNR', 'XVS', 'REEF', 'HGET', 'SFP', 'COMP', 'BRY', 'TPT',
       'BELT', 'BUNNY', 'MIR', 'BTCB', 'pCWS', 'pBTC', 'GUM', 'ODDZ',
       'JGN', 'ALPA', 'HZN', 'bROOBEE', 'mCOIN', 'MATH', 'CHR', 'WELL',
       'SWG', 'KTN', 'BAKE', 'BARK', 'TOKAU', 'BREW', 'FLP', 'ONT',
       'MOCHA', 'BETH', 'DOGE', 'APESOX', 'WEX', 'ALPACA', 'MATIC',
       'WBNB 50%', 'MEE 50%', 'SASHIMI', 'ELF', 'DOT', 'LYPTUS', 'TACO',
       'CRL', 'EGG', 'gRUPEE', 'COM+', 'HT', 'SXP', 'TABLE', 'LEG

In [24]:
pair_df

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
0,FTM,ICE,3425040.24,219.04,https://popsicle.finance,FANTOM,NaN,FARM
1,FTM,ICE,3032060.91,239.93,https://popsicle.finance,FANTOM,NaN,FARM
2,ICE,,155258.78,0.00,https://popsicle.finance,FANTOM,NaN,FARM
3,FTM,ICE,3425040.24,0.00,https://popsicle.finance,FANTOM,NaN,FARM
4,fUSDT+DAI+USDC,,27704.13,0.00,https://popsicle.finance,FANTOM,NaN,FARM
...,...,...,...,...,...,...,...,...
3846,ETH,COL,NaN,0.31628484,Uniswap,ETH,NaN,POOL
3847,ETH,DHC,NaN,191.90649480000002,Uniswap,ETH,NaN,POOL
3848,ETH,0xMR,NaN,2.4180504,Uniswap,ETH,NaN,POOL
3849,ETH,CHZ,NaN,7.4115370799999996,Uniswap,ETH,NaN,POOL


In [25]:
# pair_df['apr'].unique().tolist()

In [26]:
def convert_apr_to_apy(pair_df):
    tmp_df = pair_df[pair_df['apy'].isna()]
    print(tmp_df.shape)
    for index, row in tmp_df.iterrows():
        apr = row['apr']
        if pd.isnull( apr):
            return
        if apr:
            apr = float(apr.replace(',',''))
        else:
            apr = 0
        try:
            pair_df.loc[index, 'apy'] = ((1+(apr/100)/365)**365-1)*100
        except:
            pair_df.loc[index, 'apy'] = np.nan
    return
            
def convert_apy_to_apr(pair_df):
    tmp_df = pair_df[(pair_df['apr'].isnull())]
    print(tmp_df.shape)
    for index, row in tmp_df.iterrows():
        apy = row['apy']
        if pd.isnull( apy):
            return
        if apy:
            apy = float(apy.replace(',',''))
        else:
            apy = 0
        try:
            pair_df.loc[index, 'apr'] = (((1+(apy/100))**(1/365)-1)*365)*100
        except:
            pair_df.loc[index, 'apr'] = np.nan
    return


convert_apr_to_apy(pair_df)
convert_apy_to_apr(pair_df)

(3847, 8)
(55, 8)


In [27]:
pair_df[pair_df['apy'].isna()]

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
74,CREAM,,0.00,26607271.15,https://ftm.borgswap.exchange,FANTOM,NaN,FARM
1120,MOONTREE,sBEAR,0.00,60083983518.52,https://www.ybearswap.finance,BSC,NaN,FARM
2405,USDT,,7279.01,602183.35,https://www.bigdataprotocol.com/datavault,ETH,NaN,FARM
2406,USDC,,1022.71,4285970.71,https://www.bigdataprotocol.com/datavault,ETH,NaN,FARM
2422,USDC,,49.89,4006328.11,https://app.sirenmarkets.com/stake,ETH,NaN,FARM
...,...,...,...,...,...,...,...,...
3846,ETH,COL,NaN,0.31628484,Uniswap,ETH,NaN,POOL
3847,ETH,DHC,NaN,191.90649480000002,Uniswap,ETH,NaN,POOL
3848,ETH,0xMR,NaN,2.4180504,Uniswap,ETH,NaN,POOL
3849,ETH,CHZ,NaN,7.4115370799999996,Uniswap,ETH,NaN,POOL


In [28]:
pair_df[pair_df['apr'].isna()]

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
2768,REDUCE,,0.0,NaN,SUSHI,ETH,NaN,FARM
2803,SAFFRONCHEF,,0.0,NaN,SUSHI,ETH,NaN,FARM
2860,kmWETH,USDT,0.0,NaN,SUSHI,ETH,NaN,FARM
2861,kmWETH,USDC,0.0,NaN,SUSHI,ETH,NaN,FARM
2862,kmWETH,DAI,0.0,NaN,SUSHI,ETH,NaN,FARM
2863,kmWBTC,USDT,0.0,NaN,SUSHI,ETH,NaN,FARM
2864,kmWBTC,DAI,0.0,NaN,SUSHI,ETH,NaN,FARM
2865,kmWBTC,USDC,0.0,NaN,SUSHI,ETH,NaN,FARM
2866,kmLINK,USDT,0.0,NaN,SUSHI,ETH,NaN,FARM
2867,kmLINK,DAI,0.0,NaN,SUSHI,ETH,NaN,FARM


In [29]:
pair_df = pair_df.drop_duplicates()

In [30]:
single_piar_df = pair_df[pair_df['pair2'] == ""].copy()
double_coin_pair_df = pair_df[pair_df['pair2'] != ""].copy()
tmp_df = double_coin_pair_df.copy()

In [31]:
single_piar_df

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
2,ICE,,155258.78,0.00,https://popsicle.finance,FANTOM,0.0,FARM
4,fUSDT+DAI+USDC,,27704.13,0.00,https://popsicle.finance,FANTOM,0.0,FARM
26,BOO,,6500141.14,173.58,https://spookyswap.finance,FANTOM,465.017004,FARM
27,SPIRIT,,5042230.23,78.69,https://app.spiritswap.finance,FANTOM,119.471672,FARM
62,SON,,17306.81,195.10,https://ftm.borgswap.exchange,FANTOM,599.925881,FARM
...,...,...,...,...,...,...,...,...
2740,AGAr,,"1,678,283",197.87,https://gocerberus.finance/underworld,BSC,619.47813,FARM
2741,KRILL,,"65,451",202.95,https://gocerberus.finance/underworld,BSC,656.76194,FARM
2768,REDUCE,,0.0,NaN,SUSHI,ETH,NaN,FARM
2803,SAFFRONCHEF,,0.0,NaN,SUSHI,ETH,NaN,FARM


In [32]:
double_coin_pair_df

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
0,FTM,ICE,3425040.24,219.04,https://popsicle.finance,FANTOM,788.046372,FARM
1,FTM,ICE,3032060.91,239.93,https://popsicle.finance,FANTOM,992.931446,FARM
3,FTM,ICE,3425040.24,0.00,https://popsicle.finance,FANTOM,0.0,FARM
5,FTM,BOO,10163940.14,200.84,https://spookyswap.finance,FANTOM,641.047552,FARM
6,fUSDT,FTM,3371622.58,95.02,https://spookyswap.finance,FANTOM,158.303566,FARM
...,...,...,...,...,...,...,...,...
3846,ETH,COL,NaN,0.31628484,Uniswap,ETH,NaN,POOL
3847,ETH,DHC,NaN,191.90649480000002,Uniswap,ETH,NaN,POOL
3848,ETH,0xMR,NaN,2.4180504,Uniswap,ETH,NaN,POOL
3849,ETH,CHZ,NaN,7.4115370799999996,Uniswap,ETH,NaN,POOL


In [33]:
tmp_df

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
0,FTM,ICE,3425040.24,219.04,https://popsicle.finance,FANTOM,788.046372,FARM
1,FTM,ICE,3032060.91,239.93,https://popsicle.finance,FANTOM,992.931446,FARM
3,FTM,ICE,3425040.24,0.00,https://popsicle.finance,FANTOM,0.0,FARM
5,FTM,BOO,10163940.14,200.84,https://spookyswap.finance,FANTOM,641.047552,FARM
6,fUSDT,FTM,3371622.58,95.02,https://spookyswap.finance,FANTOM,158.303566,FARM
...,...,...,...,...,...,...,...,...
3846,ETH,COL,NaN,0.31628484,Uniswap,ETH,NaN,POOL
3847,ETH,DHC,NaN,191.90649480000002,Uniswap,ETH,NaN,POOL
3848,ETH,0xMR,NaN,2.4180504,Uniswap,ETH,NaN,POOL
3849,ETH,CHZ,NaN,7.4115370799999996,Uniswap,ETH,NaN,POOL


In [34]:
tmp_df['tmp'] = tmp_df['pair1']
tmp_df['pair1'] = tmp_df['pair2']
tmp_df['pair2'] = tmp_df['tmp']
del tmp_df['tmp']

In [35]:
double_coin_pair_df.append(tmp_df).sort_values(['pair1', 'pair2']).reset_index(drop=True)

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
0,$BASED,ETH,NaN,5.7169343999999995,Uniswap,ETH,NaN,POOL
1,$BASED,SUSD,NaN,10.6874484,Uniswap,ETH,NaN,POOL
2,$DG,ETH,738323.48,95.76,https://quickswap.exchange/#/quick,MATIC,160.217083,FARM
3,$DG,ETH,NaN,7.8732138,Uniswap,ETH,NaN,POOL
4,$DG,QUICK,308739.68,84.77,https://quickswap.exchange/#/quick,MATIC,133.197872,FARM
...,...,...,...,...,...,...,...,...
5977,zHEGIC,ETH,NaN,1.5558320399999999,Uniswap,ETH,NaN,POOL
5978,zHEGIC,HEGIC,NaN,0.8192381999999999,Uniswap,ETH,NaN,POOL
5979,zLOT,DAI,NaN,0.15694956,Uniswap,ETH,NaN,POOL
5980,zLOT,ETH,47651.84,40.69,SUSHI,ETH,NaN,FARM


In [36]:
summary = {
    "total_pairs": pair_df.shape[0],
"single_coin_pairs": single_piar_df.shape[0],
"double_coin_piars": double_coin_pair_df.shape[0],
"after_switching_coins_total_double_coin_pairs": double_coin_pair_df.shape[0] * 2,
"total_pairs_in_sheet": double_coin_pair_df.shape[0] * 2 + single_piar_df.shape[0]
}

In [37]:
summary

{'total_pairs': 3794,
 'single_coin_pairs': 803,
 'double_coin_piars': 2991,
 'after_switching_coins_total_double_coin_pairs': 5982,
 'total_pairs_in_sheet': 6785}

In [38]:
pair_df = double_coin_pair_df.append(tmp_df).sort_values(['pair1', 'pair2']).reset_index(drop=True)
pair_df = single_piar_df.append(pair_df).sort_values(['pair1', 'pair2']).reset_index(drop=True)

In [39]:
pair_df

,pair1,pair2,tvl,apr,where,network,apy,type_of_agriculture
0,$BASED,ETH,NaN,5.7169343999999995,Uniswap,ETH,NaN,POOL
1,$BASED,SUSD,NaN,10.6874484,Uniswap,ETH,NaN,POOL
2,$DG,ETH,738323.48,95.76,https://quickswap.exchange/#/quick,MATIC,160.217083,FARM
3,$DG,ETH,NaN,7.8732138,Uniswap,ETH,NaN,POOL
4,$DG,QUICK,308739.68,84.77,https://quickswap.exchange/#/quick,MATIC,133.197872,FARM
...,...,...,...,...,...,...,...,...
6780,zHEGIC,ETH,NaN,1.5558320399999999,Uniswap,ETH,NaN,POOL
6781,zHEGIC,HEGIC,NaN,0.8192381999999999,Uniswap,ETH,NaN,POOL
6782,zLOT,DAI,NaN,0.15694956,Uniswap,ETH,NaN,POOL
6783,zLOT,ETH,47651.84,40.69,SUSHI,ETH,NaN,FARM


In [40]:
pair_df['dex'] = pair_df['where']
pair_df['date'] = date 
del pair_df['where']

In [41]:
pair_df.to_csv('yield_data_2021_06_07.csv')

In [42]:
pair_df['dex'].unique().tolist()

['Uniswap',
 'https://quickswap.exchange/#/quick',
 'https://swap.cometh.io/#/stake',
 'https://caramelswap.finance',
 'https://absorber.finance',
 'https://icecreamswap.finance',
 'http://mangofarm.finance',
 'https://babyswap.finance/ilo',
 'SUSHI',
 'https://www.convexfinance.com',
 'https://ellipsis.finance',
 'https://frozenyogurt.finance',
 'https://beta.belt.fi',
 'https://blue.planetfinance.io',
 'https://polygaj.finance',
 'https://www.polycake.finance',
 'https://polylion.exchange',
 'https://www.moonwolf.io',
 'https://polygold.finance',
 'https://polypingu.finance',
 'https://polygon.prismfinance.net',
 'https://www.bigdataprotocol.com/datavault',
 'https://app.pangolin.exchange/#/png',
 'https://app.elk.finance',
 'https://spookyswap.finance',
 'https://waka.finance',
 'https://www.liquiddriver.finance',
 'https://farm.polyzap.finance',
 'https://swipe.org/farm',
 'https://koaladefi.finance/',
 'https://app.hyruleswap.com',
 'https://limeswap.io',
 'https://vanillacake.far

In [43]:
pair_df[pair_df['pair1'] == 'ETH']

,pair1,pair2,tvl,apr,network,apy,type_of_agriculture,dex,date
2657,ETH,,1065.21,108.07,FANTOM,194.204015,FARM,https://ftm.borgswap.exchange,2021-06-07
2658,ETH,,175.06,556.41,FANTOM,24916.388851,FARM,https://thegreenhouse.finance,2021-06-07
2659,ETH,,6.37,77847.71,BSC,1037842848943826568478985412663639458915456380...,FARM,https://wantanmee.finance,2021-06-07
2660,ETH,,520768.24,28.23,BSC,32.603182,FARM,https://koaladefi.finance/,2021-06-07
2661,ETH,,167226.10,46.56,BSC,59.24971,FARM,https://app.hyruleswap.com,2021-06-07
...,...,...,...,...,...,...,...,...,...
3771,ETH,yvBOOST,25943222.8,29.89,ETH,NaN,FARM,SUSHI,2021-06-07
3772,ETH,yyDAI+y...,NaN,1.9672905600000001,ETH,NaN,POOL,Uniswap,2021-06-07
3773,ETH,zHEGIC,NaN,1.5558320399999999,ETH,NaN,POOL,Uniswap,2021-06-07
3774,ETH,zLOT,NaN,0.66885036,ETH,NaN,POOL,Uniswap,2021-06-07


In [44]:
pair_df['apr'].unique().tolist()

['5.7169343999999995',
 '10.6874484',
 '95.76',
 '7.8732138',
 '84.77',
 '5.7467652000000005',
 '350.19',
 '4.15804956',
 '2.4180504',
 '0.43143084',
 '0',
 '0.00',
 '345.67',
 '291.04',
 '29.46',
 '29.85086436',
 '0.7669645199999999',
 '486.39',
 nan,
 '0.86711004',
 '2.9207509199999997',
 '3.8658324000000004',
 '17.91',
 '57.79',
 '0.14224476',
 '0.3326154',
 '76.66',
 '85.81',
 '568.48',
 '340.14',
 '560.05',
 '166350.20',
 '30.54',
 '104.34',
 '11.17',
 '3.4868001600000005',
 '73.35',
 '92.62',
 '4.79',
 '95.40',
 '101.31',
 '236.66',
 '90.26',
 '6.594082680000001',
 '3.32741472',
 '0.31',
 '37.94',
 '36.49',
 '26.31',
 '178.68',
 '155.71',
 '29.12',
 '106.17',
 '343.82',
 '52.13',
 '454.42',
 '22.36',
 '184.18',
 '112.54',
 '3.48',
 '80.31',
 '332.97',
 '117.66',
 '199.03',
 '105.83',
 '100.57',
 '231.19',
 '553.52',
 '46.18',
 '173.50',
 '79.55',
 '49.04',
 '507.18',
 '153.72',
 '265.80',
 '1,302.20',
 '143.72',
 '157.18',
 '70.76',
 '18.20',
 '261.58',
 '309.88',
 '167.15',
 '52

In [45]:
pair_df

,pair1,pair2,tvl,apr,network,apy,type_of_agriculture,dex,date
0,$BASED,ETH,NaN,5.7169343999999995,ETH,NaN,POOL,Uniswap,2021-06-07
1,$BASED,SUSD,NaN,10.6874484,ETH,NaN,POOL,Uniswap,2021-06-07
2,$DG,ETH,738323.48,95.76,MATIC,160.217083,FARM,https://quickswap.exchange/#/quick,2021-06-07
3,$DG,ETH,NaN,7.8732138,ETH,NaN,POOL,Uniswap,2021-06-07
4,$DG,QUICK,308739.68,84.77,MATIC,133.197872,FARM,https://quickswap.exchange/#/quick,2021-06-07
...,...,...,...,...,...,...,...,...,...
6780,zHEGIC,ETH,NaN,1.5558320399999999,ETH,NaN,POOL,Uniswap,2021-06-07
6781,zHEGIC,HEGIC,NaN,0.8192381999999999,ETH,NaN,POOL,Uniswap,2021-06-07
6782,zLOT,DAI,NaN,0.15694956,ETH,NaN,POOL,Uniswap,2021-06-07
6783,zLOT,ETH,47651.84,40.69,ETH,NaN,FARM,SUSHI,2021-06-07


In [46]:
pair_df[pair_df['pair1'] == 'ADA']

,pair1,pair2,tvl,apr,network,apy,type_of_agriculture,dex,date
58,ADA,,387512.79,37.94,BSC,46.111954,FARM,https://koaladefi.finance/,2021-06-07
59,ADA,,106692.53,36.49,BSC,44.010744,FARM,https://app.hyruleswap.com,2021-06-07
60,ADA,,17782.47,26.31,BSC,30.083351,FARM,https://limeswap.io,2021-06-07
61,ADA,,135707.29,178.68,BSC,494.434723,FARM,https://caramelswap.finance,2021-06-07
62,ADA,,4152.55,155.71,BSC,372.935155,FARM,https://vanillacake.farm,2021-06-07
63,ADA,,135369.54,29.12,BSC,33.787683,FARM,https://garudaswap.finance,2021-06-07
64,ADA,,748488.25,106.17,BSC,188.682958,FARM,https://gocerberus.finance,2021-06-07
65,ADA,,330916.67,343.82,BSC,2963.390885,FARM,https://ketchupfinance.com,2021-06-07
66,ADA,,22301.11,52.13,BSC,68.358945,FARM,https://TheKongFinance.com,2021-06-07
67,ADA,,"100,857",454.42,BSC,9048.227868,FARM,https://machoswap.com/farms,2021-06-07


In [48]:
df['pair1'].unique().tolist()

['ALEPH',
 'BTC',
 'COPE',
 'ETH',
 'FIDA',
 'FTT',
 'KIN',
 'LINK',
 'MAPS',
 'MEDIA',
 'MER',
 'OXY',
 'RAY',
 'ROPE',
 'SOL',
 'SRM',
 'STEP',
 'SUSHI',
 'TOMO',
 'xCOPE',
 'YFI',
 'WBTC',
 'WHALE',
 'FRONT',
 'USDT',
 'REN',
 'sUSD',
 'DAI',
 'CRO',
 'BNS',
 'SASHIMI',
 'FORTH',
 'USDC',
 'ALBT',
 'UNI',
 'LUA',
 'EASY',
 'LEND',
 'PROS',
 'BNT',
 'ARTH',
 'ANT',
 'DIA',
 'yveCRV',
 'CURA',
 'ibETH',
 'WOLF',
 'RAI',
 'TUSD',
 'SAKE',
 'XOR',
 'HEGIC',
 'FARM',
 'BUSD',
 'wCOINBASE',
 'TSD',
 'ELE',
 '1INCH',
 'MAGGOT',
 'DUCK',
 'BLZ',
 'Mars',
 'BOND',
 'BID',
 'eXRD',
 'BAT',
 'mUSO',
 'XPR',
 'PAXG',
 'BPF',
 'SOLVE',
 'LEASH',
 'BLOODY',
 'MARK',
 'UNCL',
 'DEA',
 'MOV',
 'CORE',
 'SPDR',
 'DZAR',
 'MIR',
 'ESD',
 'BACON',
 'RFI',
 'DEUS',
 'FRAX',
 'AKRO',
 'FEI',
 'TEL',
 'MKR',
 'AfroX',
 'STA',
 'SUSD',
 'RARE',
 'HAKKA',
 'DUSD',
 'OXT',
 'SALT',
 'YAM',
 'GRAIN',
 'HEX',
 'WOO',
 'XYO',
 'OPT',
 'NICE',
 'SGT',
 'POND',
 'RBC',
 'BAC',
 'YFV',
 'BONDLY',
 'MILK2',
 'FXS'